In [ ]:

from conversation_engine import ConversationEngine


def select_from_list(question, options):
    print(question)
    for i, opt in enumerate(options, start=1):
        print(f"  {i}. {opt}")
    while True:
        choice = input("> ").strip()
        if choice.isdigit():
            idx = int(choice)
            if 1 <= idx <= len(options):
                return options[idx - 1]
        print("번호로 다시 입력해 주세요.")


def run():
    print("============================================================")
    print("🌸 마음 정리 대화 시스템")
    print("============================================================\n")

    user_id = input("사용자 ID를 입력해 주세요 (예: 001): ").strip() or "test_user"
    name = input("어떻게 불러드리면 편하실까요?: ").strip() or "회원님"

    b1 = select_from_list(
        "\n요즘 마음 상태는 어떠신가요?",
        [
            "불안하다",
            "무기력하다",
            "외롭다",
            "혼란스럽다",
            "슬프다",
            "그래도 꽤 평온하다",
            "말로 표현하기 어렵다",
        ],
    )

    a4 = select_from_list(
        "\n평소 이동이나 움직임은 어느 정도 가능하신가요?",
        [
            "걷기가 비교적 편하다",
            "천천히라면 걷기는 가능하다",
            "실내에서만 주로 움직인다",
            "대부분 누워 지낸다",
        ],
    )

    profile = {"A1": name, "B1": b1, "A4": a4}
    conv = ConversationEngine(user_id=user_id, profile=profile)

    print("\n============================================================")
    print("💬 대화 시작")
    print("============================================================")
    print("💡 언제든 '그만', '끝', 'exit', '종료', '나가기' 를 입력하면 대화를 마쳐요.\n")

    welcome = conv.get_welcome_message()
    print(f"🤖 {welcome}")

    while True:
        user_msg = input("\n👤 당신: ").strip()
        if not user_msg:
            continue

        result = conv.chat(user_msg)
        ai_msg = result.get("response", "")
        print(f"\n🤖 AI: {ai_msg}")

        if result.get("end") or result.get("stage") == "END":
            break

    conv.save_session()
    print("\n💾 오늘의 대화가 저장되었어요.")
    print("============================================================")
    print("🙏 오늘도 함께해 주셔서 감사해요")
    print("============================================================\n")


if __name__ == "__main__":
    run()
